In [ ]:
import ROOT
import numpy as np
import pandas as pd
import os

In [ ]:
# Function to read in text file
def read_txt_file(file_path):
    if not os.path.exists(file_path):
        print(f"File {file_path} not found.")
        return None
    else:
        print(f"Reading in file {file_path}")
        return np.loadtxt(file_path)

In [ ]:
# Create a ROOT histogram from a numpy array
def create_TH1D_from_np_array(array, name, title, nbins, xmin, xmax):
    h = ROOT.TH1D(name, title, nbins, xmin, xmax)
    for x in array:
        h.Fill(x)
    return h

# Create a ROOT histogram from a pandas dataframe
def create_TH1D_from_pandas_df(df, column, name, title, nbins, xmin, xmax):
    h = ROOT.TH1D(name, title, nbins, xmin, xmax)
    for x in df[column]:
        h.Fill(x)
    return h

# Create a 2d ROOT histogram from a pandas dataframe

def create_TH2D_from_pandas_df(df, column1, column2, name, title, nbinsx, xmin, xmax, nbinsy, ymin, ymax):
    h = ROOT.TH2D(name, title, nbinsx, xmin, xmax, nbinsy, ymin, ymax)
    for x, y in zip(df[column1], df[column2]):
        h.Fill(x, y)
    return h


# Create a ROOT TGraph from a pandas dataframe
def create_TGraph_from_pandas_df(df, column1, column2, name, title):
    g = ROOT.TGraph(df.shape[0])
    for i, (x, y) in enumerate(zip(df[column1], df[column2])):
        g.SetPoint(i, x, y)
    return g

# Create a ROOT TGraphErrors from a pandas dataframe
def create_TGraphErrors_from_pandas_df(df, column1, column2, column3, column4, name, title):
    g = ROOT.TGraphErrors(df.shape[0])
    for i, (x, y, ex, ey) in enumerate(zip(df[column1], df[column2], df[column3], df[column4])):
        g.SetPoint(i, x, y)
        g.SetPointError(i, ex, ey)
    return g

# Create a ROOT TGraphAsymmErrors from a pandas dataframe
def create_TGraphAsymmErrors_from_pandas_df(df, column1, column2, column3, column4, column5, column6, name, title):
    g = ROOT.TGraphAsymmErrors(df.shape[0])
    for i, (x, y, exl, exh, eyl, eyh) in enumerate(zip(df[column1], df[column2], df[column3], df[column4], df[column5], df[column6])):
        g.SetPoint(i, x, y)
        g.SetPointError(i, exl, exh, eyl, eyh)
    return g

# Create a ROOT TGraph2D from a pandas dataframe
def create_TGraph2D_from_pandas_df(df, column1, column2, column3, name, title):
    g = ROOT.TGraph2D(df.shape[0])
    for i, (x, y, z) in enumerate(zip(df[column1], df[column2], df[column3])):
        g.SetPoint(i, x, y, z)
    return g

# Create a ROOT TGraph2DErrors from a pandas dataframe
def create_TGraph2DErrors_from_pandas_df(df, column1, column2, column3, column4, column5, column6, name, title):
    g = ROOT.TGraph2DErrors(df.shape[0])
    for i, (x, y, z, ex, ey, ez) in enumerate(zip(df[column1], df[column2], df[column3], df[column4], df[column5], df[column6])):
        g.SetPoint(i, x, y, z)
        g.SetPointError(i, ex, ey, ez)
    return g


# Create a ROOT TProfile from a pandas dataframe
def create_TProfile_from_pandas_df(df, column1, column2, name, title, nbinsx, xmin, xmax, ymin, ymax):
    p = ROOT.TProfile(name, title, nbinsx, xmin, xmax, ymin, ymax)
    for x, y in zip(df[column1], df[column2]):
        p.Fill(x, y)
    return p

# Create a ROOT TProfile2D from a pandas dataframe
def create_TProfile2D_from_pandas_df(df, column1, column2, column3, name, title, nbinsx, xmin, xmax, nbinsy, ymin, ymax, zmin, zmax):
    p = ROOT.TProfile2D(name, title, nbinsx, xmin, xmax, nbinsy, ymin, ymax, zmin, zmax)
    for x, y, z in zip(df[column1], df[column2], df[column3]):
        p.Fill(x, y, z)
    return p

# Create a ROOT THStack from a list of histograms
def create_THStack(h_list, name, title):
    hs = ROOT.THStack(name, title)
    for h in h_list:
        hs.Add(h)
    return hs

# Create a ROOT TLegend from a list of histograms
def create_TLegend(h_list, name, title):
    leg = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
    for h in h_list:
        leg.AddEntry(h, h.GetTitle(), "l")
    return leg

# Create a ROOT TCanvas with a list of histograms
def create_TCanvas_with_histos(h_list, name, title):
    c = ROOT.TCanvas(name, title, 800, 600)
    for i, h in enumerate(h_list):
        if i == 0:
            h.Draw()
        else:
            h.Draw("same")
    return c


In [118]:
def process_voltage(voltage):
    # Define file paths
    DATA_PATH = os.environ.get('DATA_PATH', './data')
    output_dir = "./outputs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Read files
    ch_0 = read_txt_file(f"{DATA_PATH}/{voltage}/wave_0.txt")
    tr_0 = read_txt_file(f"{DATA_PATH}/{voltage}/TR_0_0.txt")
    
    #Create a dataframe from the file
    sample_rate = 0.2 #ns
    
    # Create an empty list to store the data for each event
    data = []

    # Calculate the number of complete events (assuming all files have the same length)
    num_events = len(ch_0) // 1024
    print(f"Found {num_events} events in the data")

    event_data = {}

    for i in range(num_events):
        x = i * 1024
        event_data = {
            "Event number": i,
            "tr_0": tr_0[x:x+1024],
            "ch_0": ch_0[x:x+1024],    }
        data.append(event_data)

    # Create the DataFrame
    df = pd.DataFrame(data)

    df["tr_0_bg"] = df["tr_0"].apply(lambda x: np.mean(x[:40]))
    df["ch_0_bg"] = df["ch_0"].apply(lambda x: np.mean(x[:40]))

    df["tr_0_norm"] = df["tr_0"].apply(lambda x: x - np.mean(x[:40])) # this is the pulse with the background subtracted
    df["ch_0_norm"] = df["ch_0"].apply(lambda x: x - np.mean(x[:40])) # this is the pulse with the background subtracted

    df["tr_0_int"] = df["tr_0_norm"].apply(lambda x: np.sum(x)) # this is the integral of the pulse
    df["ch_0_int"] = df["ch_0_norm"].apply(lambda x: np.sum(x)) # this is the integral of the pulse

    df["tr_0_max"] = df["tr_0_norm"].apply(lambda x: np.max(x)) # this is the maximum of the pulse
    df["ch_0_max"] = df["ch_0_norm"].apply(lambda x: np.max(x)) # this is the maximum of the pulse

    df["tr_0_time"] = df["tr_0_norm"].apply(lambda x: np.argmax(x) * sample_rate) # this is the time of the maximum of the pulse 
    df["ch_0_time"] = df["ch_0_norm"].apply(lambda x: np.argmax(x) * sample_rate) # this is the time of the maximum of the pulse

    df["tr_0_rise"] = df["tr_0_norm"].apply(lambda x: np.argmax(x > 0.5 * np.max(x)) * sample_rate) # this is the rise time of the pulse - time to 50% of the maximum
    df["ch_0_rise"] = df["ch_0_norm"].apply(lambda x: np.argmax(x > 0.5 * np.max(x)) * sample_rate) # this is the rise time of the pulse - time to 50% of the maximum

    #subtract the rise time of the channel 0 from the rise time of the trigger
    df["rise_diff"] = df["ch_0_rise"] - df["tr_0_rise"]


    print(f"Created DataFrame with {len(df)} events and {len(df.columns)} columns")


    # Dynamically set the range for Channel 0 pulse height
    ch0_min = df["ch_0_max"].min()
    ch0_max = df["ch_0_max"].max()
    ch0_range = ch0_max - ch0_min
    ch0_plot_min = max(0, ch0_min - 0.1 * ch0_range)
    ch0_plot_max = ch0_max + 0.1 * ch0_range


    # Dynamically set the range for rise_diff
    rise_diff_min = df["rise_diff"].min()
    rise_diff_max = df["rise_diff"].max()
    rise_diff_range = rise_diff_max - rise_diff_min
    rise_diff_plot_min = rise_diff_min - 0.1 * rise_diff_range
    rise_diff_plot_max = rise_diff_max + 0.1 * rise_diff_range

    h_rise_diff = create_TH1D_from_pandas_df(df, "rise_diff", f"Rise Time Difference {voltage}V", "Rise Time Difference (ns)", 10, rise_diff_plot_min, rise_diff_plot_max)
    h2d_rise_diff = create_TH2D_from_pandas_df(df, "ch_0_max", "rise_diff", f"Channel 0 Pulse Height vs Rise Time Difference {voltage}V", "Channel 0;Pulse Height;Rise Time Difference (ns)", 10, ch0_plot_min, ch0_plot_max, 10, rise_diff_plot_min, rise_diff_plot_max)

    # Create histograms
    h_tr_pulse = create_TH1D_from_pandas_df(df, "tr_0_max", f"Trigger Pulse Height Spectrum {voltage}V", "Trigger Pulse Height", 50, 3900, 4200)
    h_ch0_pulse = create_TH1D_from_pandas_df(df, "ch_0_max", f"Channel 0 Pulse Height Spectrum {voltage}V", "Channel 0 Pulse Height", 50, ch0_plot_min, ch0_plot_max)

    h_tr_tdc = create_TH1D_from_pandas_df(df, "tr_0_rise", f"Trigger TDC Spectrum {voltage}V", "Trigger Rise Time", 50, 8, 20)
    h_ch0_tdc = create_TH1D_from_pandas_df(df, "ch_0_rise", f"Channel 0 TDC Spectrum {voltage}V", "Channel 0 Rise Time", 50, 15, 28)

    h2d_tr = create_TH2D_from_pandas_df(df, "tr_0_max", "tr_0_rise", f"Trigger Pulse Height vs Rise Time {voltage}V", "Trigger;Pulse Height;Rise Time (ns)", 50, 3900, 4200, 50, 8, 20)
    h2d_ch0 = create_TH2D_from_pandas_df(df, "ch_0_max", "ch_0_rise", f"Channel 0 Pulse Height vs Rise Time {voltage}V", "Channel 0;Pulse Height;Rise Time (ns)", 50, ch0_plot_min, ch0_plot_max, 50, 15, 28)

    #save the histograms to a root file
    f = ROOT.TFile(f"{output_dir}/histograms_{voltage}V.root", "RECREATE")
    h_tr_pulse.Write()
    h_ch0_pulse.Write()
    h_tr_tdc.Write()
    h_ch0_tdc.Write()
    h2d_tr.Write()
    h2d_ch0.Write()
    h_rise_diff.Write()
    h2d_rise_diff.Write()
    f.Close()

    # Draw and save histograms
    c1 = ROOT.TCanvas("c1", f"Pulse Height Spectra {voltage}V", 800, 400)
    c1.Divide(2,1)
    c1.cd(1)
    h_tr_pulse.Draw()
    h_tr_pulse.GetXaxis().SetTitle("Pulse Height")
    h_tr_pulse.GetYaxis().SetTitle("Counts")
    c1.cd(2)
    h_ch0_pulse.Draw()
    h_ch0_pulse.GetXaxis().SetTitle("Pulse Height")
    h_ch0_pulse.GetYaxis().SetTitle("Counts")
    c1.SaveAs(os.path.join(output_dir, f"pulse_height_spectra_{voltage}V.png"))

    c2 = ROOT.TCanvas("c2", f"TDC Spectra {voltage}V", 800, 400)
    c2.Divide(2,1)
    c2.cd(1)
    h_tr_tdc.Draw()
    h_tr_tdc.GetXaxis().SetTitle("Rise Time (ns)")
    h_tr_tdc.GetYaxis().SetTitle("Counts")
    c2.cd(2)
    h_ch0_tdc.Draw()
    h_ch0_tdc.GetXaxis().SetTitle("Rise Time (ns)")
    h_ch0_tdc.GetYaxis().SetTitle("Counts")
    c2.SaveAs(os.path.join(output_dir, f"tdc_spectra_{voltage}V.png"))

    c3 = ROOT.TCanvas("c3", f"Pulse Height vs TDC {voltage}V", 800, 400)
    c3.Divide(2,1)
    c3.cd(1)
    h2d_tr.Draw("COLZ")
    c3.cd(2)
    h2d_ch0.Draw("COLZ")
    c3.SaveAs(os.path.join(output_dir, f"pulse_height_vs_tdc_{voltage}V.png"))

    c4 = ROOT.TCanvas("c4", f"Rise Time Difference {voltage}V", 800, 400)
    c4.Divide(2,1)
    c4.cd(1)
    h_rise_diff.Draw()
    h_rise_diff.GetXaxis().SetTitle("Rise Time Difference (ns)")
    h_rise_diff.GetYaxis().SetTitle("Counts")
    c4.cd(2)
    h2d_rise_diff.Draw("COLZ")
    h2d_rise_diff.GetXaxis().SetTitle("Channel 0 Pulse Height")
    h2d_rise_diff.GetYaxis().SetTitle("Rise Time Difference (ns)")
    c4.SaveAs(os.path.join(output_dir, f"rise_time_difference_{voltage}V.png"))

    # Print summary information
    print("---")
    print(f"Voltage: {voltage}V")
    print(f"Channel 0 Pulse Height Range: {ch0_min:.2f} to {ch0_max:.2f}")
    print(f"Plotting Range: {ch0_plot_min:.2f} to {ch0_plot_max:.2f}")
    print("---")


    # After calculating tr_0_rise and ch_0_rise, add:
    df["rise_diff"] = df["ch_0_rise"] - df["tr_0_rise"]

    # Print some statistics about rise_diff
    print(f"Rise Time Difference Statistics for {voltage}V:")
    print(df["rise_diff"].describe())
    print("Min:", df["rise_diff"].min())
    print("Max:", df["rise_diff"].max())
    print("---")




In [119]:
for voltage in [25, 26, 27]:
    process_voltage(voltage)
    

Reading in file ./data/25/wave_0.txt
Reading in file ./data/25/TR_0_0.txt
Found 14321 events in the data
Created DataFrame with 14321 events and 16 columns
---
Voltage: 25V
Channel 0 Pulse Height Range: 182.22 to 228.45
Plotting Range: 177.59 to 233.08
---
Rise Time Difference Statistics for 25V:
count    14321.000000
mean         6.688388
std          0.101276
min          6.200000
25%          6.600000
50%          6.600000
75%          6.800000
max          7.000000
Name: rise_diff, dtype: float64
Min: 6.200000000000001
Max: 7.0
---
Reading in file ./data/26/wave_0.txt
Reading in file ./data/26/TR_0_0.txt
Found 12276 events in the data
Created DataFrame with 12276 events and 16 columns
---
Voltage: 26V
Channel 0 Pulse Height Range: 1188.34 to 1338.20
Plotting Range: 1173.35 to 1353.19
---
Rise Time Difference Statistics for 26V:
count    12276.000000
mean         6.532209
std          0.097222
min          6.200000
25%          6.400000
50%          6.600000
75%          6.600000
ma

Info in <TCanvas::Print>: png file ./outputs/pulse_height_spectra_25V.png has been created
Info in <TCanvas::Print>: png file ./outputs/tdc_spectra_25V.png has been created
Info in <TCanvas::Print>: png file ./outputs/pulse_height_vs_tdc_25V.png has been created
Info in <TCanvas::Print>: png file ./outputs/rise_time_difference_25V.png has been created
Info in <TCanvas::Print>: png file ./outputs/pulse_height_spectra_26V.png has been created
Info in <TCanvas::Print>: png file ./outputs/tdc_spectra_26V.png has been created
Info in <TCanvas::Print>: png file ./outputs/pulse_height_vs_tdc_26V.png has been created
Info in <TCanvas::Print>: png file ./outputs/rise_time_difference_26V.png has been created
Info in <TCanvas::Print>: png file ./outputs/pulse_height_spectra_27V.png has been created
Info in <TCanvas::Print>: png file ./outputs/tdc_spectra_27V.png has been created
Info in <TCanvas::Print>: png file ./outputs/pulse_height_vs_tdc_27V.png has been created
Info in <TCanvas::Print>: png